In [2]:
%pip install torchvision
%pip install matplotlib
%pip install opencv-python

  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.3 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/197.9 MB ? eta -:--:--
   ---------------------------------------- 1.6/197.9 MB 34.2 MB/s eta 0:00:06
    --------------------------------------- 4.0/197.9 MB 42.5 MB/s eta 0:00:05
   - -------------------------------------- 5.1/197.9 MB 40.4 MB/s eta 0:00:05
   - -------------------------------------- 7.9/197.9 MB 45.9 MB/s eta 0:


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\daneil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\daneil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\daneil\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [4]:
# Configura o dispositivo para CPU
device = 'cpu'

In [5]:
# Cria um tensor de entrada X_train com valores float e move para o dispositivo especificado
X_train = torch.tensor([[
    [[4,5,6,7],[5,6,7,8],[8,9,10,11],[4,5,6,7]]],
    [[[-4,5,6,-7],[5,-6,7,8],[-8,-9,-10,11],[-4,-6,-7,-8]]]                    
]).float().to(device)


In [6]:
# Normaliza X_train dividindo por 8
X_train.div_(8)


tensor([[[[ 0.5000,  0.6250,  0.7500,  0.8750],
          [ 0.6250,  0.7500,  0.8750,  1.0000],
          [ 1.0000,  1.1250,  1.2500,  1.3750],
          [ 0.5000,  0.6250,  0.7500,  0.8750]]],


        [[[-0.5000,  0.6250,  0.7500, -0.8750],
          [ 0.6250, -0.7500,  0.8750,  1.0000],
          [-1.0000, -1.1250, -1.2500,  1.3750],
          [-0.5000, -0.7500, -0.8750, -1.0000]]]])

In [7]:
# Cria o tensor de rótulos y_train com valores float e move para o dispositivo especificado
y_train = torch.tensor([0,1]).float().to(device)


In [8]:

# Exibe as dimensões de X_train e y_train
print(X_train.shape)  # torch.Size([2, 1, 4, 4])
print(y_train.shape)  # torch.Size([2])

torch.Size([2, 1, 4, 4])
torch.Size([2])


In [9]:

def get_model():
    """
    Cria um modelo de rede neural com uma camada convolucional, uma camada de pooling,
    uma função de ativação ReLU, uma camada de flattening e uma camada linear com sigmoid.
    """
    model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3),  # Camada convolucional
        nn.MaxPool2d(kernel_size=2),  # Camada de pooling
        nn.ReLU(),  # Função de ativação ReLU
        nn.Flatten(),  # Achata a saída para um vetor unidimensional
        nn.Linear(1, 1),  # Camada totalmente conectada
        nn.Sigmoid()  # Função de ativação Sigmoid para obter a probabilidade
    ).to(device)  # Move o modelo para o dispositivo especificado

    # Define a função de perda como Binary Cross Entropy
    loss_fn = nn.BCELoss()
    
    # Define o otimizador Adam com taxa de aprendizado de 0.01
    optimizer = Adam(model.parameters(), lr=0.01)

    return model, loss_fn, optimizer

# Obtém o modelo, a função de perda e o otimizador
model, criterion, optimizer = get_model()
print(model)

Sequential(
  (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): ReLU()
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=1, out_features=1, bias=True)
  (5): Sigmoid()
)


In [10]:

def train_batch(x, y, model, optimizer, loss_fn):
    """
    Treina o modelo em um lote de dados, atualiza os pesos e retorna a perda do lote.
    """
    model.train()  # Define o modelo em modo de treinamento
    optimizer.zero_grad()  # Zera os gradientes do otimizador
    prediction = model(x)  # Faz a previsão com o modelo
    batch_loss = loss_fn(prediction.squeeze(), y.squeeze())  # Calcula a perda
    batch_loss.backward()  # Calcula os gradientes
    optimizer.step()  # Atualiza os pesos do modelo
    return batch_loss.item()  # Retorna o valor da perda como um número

# Cria um DataLoader para carregar os dados em lotes
trn_dl = DataLoader(TensorDataset(X_train, y_train))


In [11]:

# Treina o modelo por 2000 épocas
for e in range(2000):
    for ix, batch in enumerate(trn_dl):
        x, y = batch
        x = x.to(device)  # Move os dados para o dispositivo
        y = y.to(device)  # Move os rótulos para o dispositivo
        batch_loss = train_batch(x, y, model, optimizer, criterion)  # Treina o modelo no lote

# Faz uma previsão com o modelo treinado
print(model(X_train[:1]))  # Exibe a saída para o primeiro exemplo do conjunto de treinamento

tensor([[0.0036]], grad_fn=<SigmoidBackward0>)
